In [1]:
import pandas as pd
import yfinance as yf
from datetime import date
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/assets july 7 2025.csv")
df.head()

,Asset,Ticker,Units,Purchase Price,Currency Purchase,Currency Yahoo,Price Last Update,Date Last Update,Value Last Update,Profit Last Update
0,S&P 500 ETF,SXR8.DE,17.4780,584.85,EUR,EUR,560.36,2025-06-30,9793.972080,-428.036220
1,Zalando,ZAL.DE,407.0000,44.08,EUR,EUR,28.16,2025-06-30,11461.120000,-6479.440000
2,S&P 500 IT ETF,QDVE.DE,340.7460,30.68,EUR,EUR,30.85,2025-06-30,10512.014100,57.926820
3,Bitcoin,BTC-USD,0.1279,44894.00,EUR,USD,91958.00,2025-06-30,11761.428200,6019.485600
4,Microsoft,MSFT,0.0389,384.06,EUR,USD,422.55,2025-06-30,16.437195,1.497261


In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Asset               5 non-null      str    
 1   Ticker              5 non-null      str    
 2   Units               5 non-null      float64
 3   Purchase Price      5 non-null      float64
 4   Currency Purchase   5 non-null      str    
 5   Currency Yahoo      5 non-null      str    
 6   Price Last Update   5 non-null      float64
 7   Date Last Update    5 non-null      str    
 8   Value Last Update   5 non-null      float64
 9   Profit Last Update  5 non-null      float64
dtypes: float64(5), str(5)
memory usage: 532.0 bytes


# FX Rate

In [5]:
fx_pair = 'USDEUR=X'  # Yahoo Finance symbol for USD to EUR
fx_data = yf.Ticker(fx_pair).history(period="1d")
fx_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2026-02-13 00:00:00+00:00,0.8423,0.8438,0.8413,0.8423,0,0.0,0.0


In [10]:
# Fetch FX rate for USD → EUR
def get_fx_rate(from_currency: str, to_currency: str = 'EUR') -> float:
    """
    Get live FX rate from Yahoo. Returns 1.0 if from/to are the same.
    """
    if from_currency == to_currency:
        return 1.0
    pair = f"{from_currency}{to_currency}=X"
    try:
        data = yf.Ticker(pair).history(period="1d")
        return data["Close"].iloc[-1]
    except Exception as e:
        print(f"Error fetching FX rate: {e}")
        return np.nan

In [11]:
# test the get_fx_rate
print(get_fx_rate('USD', 'EUR'))

0.8422999978065491


In [12]:
# Cache FX rates with EUR = 1.0
fx_cache = {}


for ccy in df['Currency Yahoo'].unique():
    if ccy == 'EUR':
        fx_cache[ccy] = 1.0  # no conversion needed
    else:
        fx_cache[ccy] = get_fx_rate(ccy)
print("FX Cache:", fx_cache)

FX Cache: {'EUR': 1.0, 'USD': np.float64(0.8424999713897705)}


In [13]:
# Download one price from a ticker for a specific date
yf.Ticker('BTC-USD').history(period="1d")['Close']

Date
2026-02-13 00:00:00+00:00    68784.46875
Name: Close, dtype: float64

In [14]:
def get_price_eur(row) -> float:
    """
    Fetch Yahoo price in native currency and convert to EUR.
    """
    try:
        price_native = (
            yf.Ticker(row['Ticker'])
              .history(period="1d")['Close']
              .iloc[-1]
        )
        fx_rate = fx_cache.get(row['Currency Yahoo'], 1.0)  # default 1.0
        return price_native * fx_rate
    except Exception as e:
        print(f"Error fetching {row['Ticker']}: {e}")
        return np.nan

In [15]:
# get the price for all tickers
df.apply(get_price_eur, axis=1)

0      619.359985
1       20.510000
2       33.555000
3    57923.920045
4      338.718696
dtype: float64